ConvLSTM trained on gridded forcings and landcover data for all stations.
Test generalization by training on a subset of stations

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn, utils
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, conv_lstm, datasets
import torch.autograd as autograd
import pickle

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190727-101522'

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../log.out', mode='a')
chandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - {} - %(message)s'.format(time_stamp))
fhandler.setFormatter(formatter)
chandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.INFO)

In [3]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
logger.warning('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)

CUDA Available
2019-07-27 10:15:22,950 - 20190727-101522 - cuda devices: ['Tesla V100-SXM2-16GB']


In [4]:
seq_len = 5*24
seq_steps = 2
stateful_lstm = False
validation_fraction, val_start, val_end = None, None, None
test_station_fraction = 0.25

if stateful_lstm:
    val_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(hours=seq_len * seq_steps)  # first day for which to make a prediction in train set
    val_end = '2010-09-30'
    train_start = '2010-10-01'
    train_end = '2012-12-31'
else:
    validation_fraction = 0.1
    train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(hours=seq_len * seq_steps)  # first day for which to make a prediction in train set
    train_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'

In [5]:
rdrs_vars = list(range(8))
train_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, train_start, train_end)
if stateful_lstm:
    val_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, val_start, val_end, conv_scalers=train_dataset.conv_scalers)
test_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, test_start, test_end, conv_scalers=train_dataset.conv_scalers)

../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]
../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]


In [6]:
landcover_types = None
landcover, landcover_legend = load_data.load_landcover_reduced(values_to_use=landcover_types)
landcover = torch.from_numpy(landcover).float().to(device)

../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]
../src/load_data.py:242: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  landcover_fullres = np.array(landcover_nc['Band1'][:])[::-1,:]


In [7]:
stations = train_dataset.data_runoff['station'].unique()
num_test_stations = int(test_station_fraction * len(stations))
test_stations = np.random.choice(stations, size=num_test_stations, replace=False)
train_stations = list(s for s in stations if s not in test_stations)

train_station_indices = list(train_dataset.station_to_index[s] for s in train_stations)
test_station_indices = list(test_dataset.station_to_index[s] for s in test_stations)

train_mask = torch.zeros((train_dataset.conv_height, train_dataset.conv_width), dtype=torch.int8)
test_mask = torch.zeros((train_dataset.conv_height, train_dataset.conv_width), dtype=torch.int8)
for row in range(train_dataset.conv_height):
    for col in range(train_dataset.conv_width):
        train_mask[row, col] = 1 if (row, col) in train_station_indices else 0
        test_mask[row, col] = 1 if (row, col) in test_station_indices else 0
train_mask = train_mask.reshape(-1).to(device)
test_mask = test_mask.reshape(-1).to(device)

In [8]:
# Train model
num_epochs = 400
learning_rate = 2e-3
patience = 100
min_improvement = 0.01
best_loss_model = (-1, np.inf, None)

# Prepare model
batch_size = 32
num_convlstm_layers = 2
num_conv_layers = 2
convlstm_hidden_dims = [8] * num_convlstm_layers
conv_hidden_dims = [4] * (num_conv_layers - 1)
convlstm_kernel_size = [(5,5)] * num_convlstm_layers
conv_kernel_size = [(3,3)] * num_conv_layers
conv_activation = nn.Sigmoid
dropout = 0.2
weight_decay = 1e-5

model = conv_lstm.ConvLSTMGridWithGeophysicalInput((train_dataset.conv_height, train_dataset.conv_width), 
                                         train_dataset.n_conv_vars, landcover.shape[0], convlstm_hidden_dims, 
                                         conv_hidden_dims, convlstm_kernel_size, conv_kernel_size, 
                                         num_convlstm_layers, num_conv_layers, conv_activation, dropout=dropout).to(device)
model = torch.nn.DataParallel(model, device_ids=list(range(num_devices)))
loss_fn = evaluate.NSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

writer = SummaryWriter()
param_description = {'time_stamp': time_stamp, 'H_convlstm': convlstm_hidden_dims, 'H_conv': conv_hidden_dims, 'batch_size': batch_size, 'num_convlstm_layers': num_convlstm_layers, 'num_conv_layers': num_conv_layers, 'convlstm_kernel_size': convlstm_kernel_size, 'conv_kernel_size': conv_kernel_size, 'loss': loss_fn, 
                     'optimizer': optimizer, 'lr': learning_rate, 'patience': patience, 'min_improvement': min_improvement, 'stateful_lstm': stateful_lstm, 'dropout': dropout, 'landcover_shape': landcover.shape, 'conv_activation': conv_activation, 'test_station_fraction': test_station_fraction,
                     'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 'train_start': train_start, 'train_end': train_end, 'weight_decay': weight_decay, 'validation_fraction': validation_fraction, 'landcover_types': landcover_types,
                     'test_start': test_start, 'test_end': test_end, 'n_conv_vars': train_dataset.n_conv_vars, 'model': str(model).replace('\n','').replace(' ', ''), 'val_start': val_start, 'val_end': val_end, 'train_stations': train_stations, 'test_stations': test_stations,
                     'train len':len(train_dataset), 'conv_height': train_dataset.conv_height, 'conv_width': train_dataset.conv_width, 'test len': len(test_dataset)}
writer.add_text('Parameter Description', str(param_description))
str(param_description)

"{'time_stamp': '20190727-101522', 'H_convlstm': [8, 8], 'H_conv': [4], 'batch_size': 32, 'num_convlstm_layers': 2, 'num_conv_layers': 2, 'convlstm_kernel_size': [(5, 5), (5, 5)], 'conv_kernel_size': [(3, 3), (3, 3)], 'loss': NSELoss(), 'optimizer': Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    eps: 1e-08\n    lr: 0.002\n    weight_decay: 1e-05\n), 'lr': 0.002, 'patience': 100, 'min_improvement': 0.01, 'stateful_lstm': False, 'dropout': 0.2, 'landcover_shape': torch.Size([19, 34, 39]), 'conv_activation': <class 'torch.nn.modules.activation.Sigmoid'>, 'test_station_fraction': 0.25, 'num_epochs': 400, 'seq_len': 120, 'seq_steps': 2, 'train_start': datetime.datetime(2010, 1, 11, 0, 0), 'train_end': '2012-12-31', 'weight_decay': 1e-05, 'validation_fraction': 0.1, 'landcover_types': None, 'test_start': '2013-01-01', 'test_end': '2014-12-31', 'n_conv_vars': 8, 'model': 'DataParallel((module):ConvLSTMGridWithGeophysicalInput((conv_lstm):ConvLSTM((cell_list):Modul

In [9]:
if stateful_lstm:
    train_sampler = datasets.StatefulBatchSampler(train_dataset, batch_size)
    val_sampler = datasets.StatefulBatchSampler(val_dataset, batch_size)
    test_sampler = datasets.StatefulBatchSampler(test_dataset, batch_size)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_sampler, pin_memory=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_sampler=val_sampler, pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_sampler, pin_memory=True)
else:
    val_indices = np.random.choice(len(train_dataset), size=int(validation_fraction * len(train_dataset)), replace=False)
    train_indices = list(i for i in range(len(train_dataset)) if i not in val_indices)
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=train_sampler, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=val_sampler, pin_memory=True, drop_last=False)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)

In [10]:
torch.manual_seed(0)
np.random.seed(0)
for epoch in range(num_epochs):
    model.train()

    epoch_losses = torch.tensor(0.0)
    conv_hidden_states = None
    for i, train_batch in enumerate(train_dataloader):
        y_train = train_batch['y'].reshape((train_batch['y'].shape[0],-1)).to(device, non_blocking=True)
        mask = train_batch['mask'].sum(dim=0, dtype=torch.int8).reshape(-1).to(device, non_blocking=True) != 0
        mask = (mask.to(torch.int8) + train_mask) > 1
        landcover_batch = landcover.repeat(y_train.shape[0],1,1,1)
        
        if mask.sum() == 0:
            print('Batch {} has no target values. skipping.'.format(i))
            continue
        if not stateful_lstm:
            conv_hidden_states = None
        
        y_pred, conv_hidden_states = model(train_batch['x_conv'].to(device), landcover_batch, hidden_state=conv_hidden_states)
        y_pred = y_pred.reshape((train_batch['y'].shape[0], -1))
        loss = loss_fn(y_pred[:,mask], y_train[:,mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_losses += loss.detach()

    epoch_loss = (epoch_losses / len(train_dataloader)).item()
    print('Epoch', epoch, 'mean train loss:\t{}'.format(epoch_loss))
    writer.add_scalar('loss_nse', epoch_loss, epoch)
    
    # eval on validation split
    model.eval()
    val_losses = torch.tensor(0.0)
    conv_hidden_states = None
    for i, val_batch in enumerate(val_dataloader):
        y_val = val_batch['y'].reshape((val_batch['y'].shape[0],-1)).to(device, non_blocking=True)
        mask = val_batch['mask'].sum(dim=0, dtype=torch.int8).reshape(-1).to(device, non_blocking=True) != 0
        mask = (mask.to(torch.int8) + train_mask) > 0
        landcover_batch = landcover.repeat(y_val.shape[0],1,1,1)
        
        if not stateful_lstm:
            conv_hidden_states = None
        
        batch_pred, conv_hidden_states = model(val_batch['x_conv'], landcover_batch, hidden_state=conv_hidden_states)
        batch_pred = batch_pred.detach().reshape((val_batch['y'].shape[0], -1))
        val_losses += loss_fn(batch_pred[:,mask], y_val[:,mask]).detach()

    val_nse = (val_losses / len(val_dataloader)).item()
    print('Epoch {} mean val loss:  \t{}'.format(epoch, val_nse))
    writer.add_scalar('loss_nse_val', val_nse, epoch)
    if val_nse < best_loss_model[1] - min_improvement:
        best_loss_model = (epoch, val_nse, model.state_dict())  # new best model
        load_data.pickle_model('ConvLSTM_withLandcover_generalizationTest', model, 'allStations', time_stamp)
    elif epoch > best_loss_model[0] + patience:
        print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
        break

print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
model.load_state_dict(best_loss_model[2])
load_data.pickle_model('ConvLSTM_withLandcover_generalizationTest', model, 'allStations', time_stamp)

Epoch 0 mean train loss:	1.4346424341201782
Epoch 0 mean val loss:  	1.4885663986206055
Saved model as ../pickle/models/ConvLSTM_withLandcover_generalizationTest_allStations_20190727-101522.pkl
Epoch 1 mean train loss:	1.2579255104064941
Epoch 1 mean val loss:  	1.2850016355514526
Saved model as ../pickle/models/ConvLSTM_withLandcover_generalizationTest_allStations_20190727-101522.pkl
Epoch 2 mean train loss:	1.1558414697647095
Epoch 2 mean val loss:  	1.2063175439834595
Saved model as ../pickle/models/ConvLSTM_withLandcover_generalizationTest_allStations_20190727-101522.pkl
Epoch 3 mean train loss:	1.1054607629776
Epoch 3 mean val loss:  	1.081693172454834
Saved model as ../pickle/models/ConvLSTM_withLandcover_generalizationTest_allStations_20190727-101522.pkl
Epoch 4 mean train loss:	1.0057495832443237
Epoch 4 mean val loss:  	0.9569349884986877
Saved model as ../pickle/models/ConvLSTM_withLandcover_generalizationTest_allStations_20190727-101522.pkl
Epoch 5 mean train loss:	0.9937777

In [11]:
logger.warning('predicting')
model.eval()

predictions = []
conv_hidden_states = None
for i, test_batch in enumerate(test_dataloader):
    if not stateful_lstm:
        conv_hidden_states = None
        
    landcover_batch = landcover.repeat(test_batch['y'].shape[0],1,1,1)
    pred, conv_hidden_states = model(test_batch['x_conv'], landcover_batch, hidden_state=conv_hidden_states)
    predictions.append(pred.detach())

predictions = torch.cat(predictions).cpu()

if stateful_lstm:
    # reorder time series
    pred_indices = np.array(list(test_sampler.__iter__())).reshape(-1)
    predictions = predictions[pred_indices.argsort()]

2019-07-27 11:13:43,054 - 20190727-101522 - predicting


In [12]:
actuals = test_dataset.data_runoff.copy()
if len(actuals['date'].unique()) != len(predictions):
    print('Warning: length of prediction {} and actuals {} does not match.'.format(len(predictions), len(actuals['date'].unique())))

nse_dict = {}
mse_dict = {}
predictions_df = pd.DataFrame(columns=actuals.columns)
predictions_df['is_test_station'] = False
for station in actuals['station'].unique():
    row, col = test_dataset.station_to_index[station]
    
    act = actuals[actuals['station'] == station].set_index('date')['runoff']
    if predictions.shape[0] != act.shape[0]:
        print('Warning: length of prediction {} and actuals {} does not match for station {}. Ignoring excess actuals.'.format(len(predictions), len(act), station))
        act = act.iloc[:predictions.shape[0]]
    pred = pd.DataFrame({'runoff': predictions[:,row,col]}, index=act.index)
    pred['station'] = station
    pred['is_test_station'] = station in test_stations
    predictions_df = predictions_df.append(pred.reset_index(), sort=True)
    
    nse, mse = evaluate.evaluate_daily(station, pred['runoff'], act, writer=writer)
    nse_dict[station] = nse
    mse_dict[station] = mse
    
    print(station, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')

/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


02GA047 	NSE: 0.3280552511787559 	MSE: 52.67376543338388 (clipped to 0)
04213000 	NSE: 0.41489921915855865 	MSE: 105.00759034478574 (clipped to 0)
04176500 	NSE: 0.12885220313411794 	MSE: 595.0585986893636 (clipped to 0)
02GG003 	NSE: 0.3225439570453644 	MSE: 323.4159370010082 (clipped to 0)
04214500 	NSE: 0.4095079371421213 	MSE: 71.17619318599607 (clipped to 0)
02GC026 	NSE: 0.40385619450249644 	MSE: 95.87944962570282 (clipped to 0)
04174500 	NSE: -1.1333470438448154 	MSE: 201.75966804420116 (clipped to 0)
02GG013 	NSE: 0.02235217892845387 	MSE: 37.328920732711495 (clipped to 0)
04161820 	NSE: -0.9355906462481465 	MSE: 61.122176486288666 (clipped to 0)
04159492 	NSE: 0.30536949654683376 	MSE: 322.51263795315384 (clipped to 0)
04200500 	NSE: 0.4406773202163633 	MSE: 559.7169776110782 (clipped to 0)
02GB001 	NSE: -0.354597416056069 	MSE: 10172.857538930497 (clipped to 0)
04208504 	NSE: 0.4962272089155052 	MSE: 491.94856669434506 (clipped to 0)
04199000 	NSE: 0.40462037429147835 	MSE: 6

In [13]:
nse_train = list(nse_dict[s] for s in train_stations)
mse_train = list(mse_dict[s] for s in train_stations)
print('Train Median NSE (clipped to 0)', np.median(nse_train), '/ Min', np.min(nse_train), '/ Max', np.max(nse_train))
print('Train Median MSE (clipped to 0)', np.median(mse_train), '/ Min', np.min(mse_train), '/ Max', np.max(mse_train))

nse_test = list(nse_dict[s] for s in test_stations)
mse_test = list(mse_dict[s] for s in test_stations)
print('Test Median NSE (clipped to 0)', np.median(nse_test), '/ Min', np.min(nse_test), '/ Max', np.max(nse_test))
print('Test Median MSE (clipped to 0)', np.median(mse_test), '/ Min', np.min(mse_test), '/ Max', np.max(mse_test))

writer.add_scalar('nse_median', np.median(nse_test))

Train Median NSE (clipped to 0) 0.37047758246860063 / Min -0.3697806146063056 / Max 0.5603641288277227
Train Median MSE (clipped to 0) 105.00759034478574 / Min 6.266778075126123 / Max 96981.49384668478
Test Median NSE (clipped to 0) 0.31133489657275226 / Min -27.107050397290227 / Max 0.4089291701112878
Test Median MSE (clipped to 0) 166.09419610663642 / Min 34.85623816398938 / Max 1518.1187322565186


In [14]:
save_df = pd.merge(predictions_df.rename({'runoff': 'prediction'}, axis=1), actuals.rename({'runoff': 'actual'}, axis=1), 
                   on=['date', 'station'])[['date', 'station', 'prediction', 'actual', 'is_test_station']]
load_data.pickle_results('ConvLSTM_withLandcover_generalizationTest', save_df, time_stamp)

'ConvLSTM_withLandcover_generalizationTest_20190727-101522.pkl'

In [15]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190727-111354'